In [1]:
from pymongo import MongoClient
import pandas as pd
import time
import json

In [2]:
client = MongoClient("localhost:27017")

In [3]:
client.list_database_names()
db = client["Ironhack"]

In [4]:
db.list_collection_names()

['PROJECT3', 'nyneigh', 'First', 'Lab', 'nyrest']

In [5]:
companys= db.get_collection("Lab")

In [ ]:
companys

In [6]:
result = companys.find_one()

In [7]:
df = pd.read_csv('Lab.csv')

In [8]:
df.sample(2)

,_id,acquisition,acquisition.acquired_day,acquisition.acquired_month,acquisition.acquired_year,acquisition.acquiring_company,acquisition.price_amount,acquisition.price_currency_code,acquisition.source_description,acquisition.source_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
584,52cdef7d4bab8bd675299e8f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,[],[],[],[],"fantasy-sports, game",$0,NaN,Tue Apr 28 11:24:27 UTC 2009,[]
890,52cdef7e4bab8bd67529b4c0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"[{""name"":""Howrse"",""permalink"":""howrse""},{""name...","[{""title"":""France & Germany"",""is_past"":true,""p...","[{""is_past"":false,""title"":""CEO"",""person"":{""fir...","[{""available_sizes"":[[[150,84],""assets/images/...","game, hobby, browser, virtual-goods",€3M,owlient,Tue Apr 02 11:42:49 UTC 2013,[]


In [9]:
data = pd.read_json ('Labjson.csv')

In [10]:
data.sample(5)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,providerships,total_money_raised,funding_rounds,investments,offices,video_embeds,screenshots,external_links,partners,ipo
106,{'$oid': '52cdef7c4bab8bd6752980f3'},Dabble,dabble,http://www.crunchbase.com/company/dabble,http://dabble.com,http://dabble.com/blog,http://dabble.com/blog/feed/,None,games_video,NaN,...,[],$350k,"[{'id': 1003, 'round_code': 'seed', 'source_ur...",[],"[{'description': None, 'address1': '2342 Shatt...",[],[],[],[],NaN
475,{'$oid': '52cdef7d4bab8bd67529959e'},DeNA,dena,http://www.crunchbase.com/company/dena,http://dena.com,,,,games_video,2192.0,...,"[{'title': '', 'is_past': False, 'provider': {...",¥2.21B,"[{'id': 18915, 'round_code': 'a', 'source_url'...",[{'funding_round': {'round_code': 'unattribute...,"[{'description': 'Headquarters', 'address1': '...",[],[],[],[],{'valuation_amount': 117900}
127,{'$oid': '52cdef7c4bab8bd67529820b'},iVideosongs,ivideosongs,http://www.crunchbase.com/company/ivideosongs,http://www.ivideosongs.com,,None,None,games_video,NaN,...,[],$3.3M,"[{'id': 20977, 'round_code': 'angel', 'source_...",[],"[{'description': None, 'address1': None, 'addr...",[],[],[],[],NaN
450,{'$oid': '52cdef7d4bab8bd675299421'},ngmoco,ngmoco,http://www.crunchbase.com/company/ngmoco,http://www.ngmoco.com,http://blog.ngmoco.com,http://blog.ngmoco.com/rss,ngmoco,games_video,150.0,...,[],$44.8M,"[{'id': 3066, 'round_code': 'a', 'source_url':...",[],"[{'description': 'Secret Factory', 'address1':...","[{'embed_code': '<object width=""480"" height=""3...",[],[],[],NaN
537,{'$oid': '52cdef7d4bab8bd675299a73'},Bojam,bojam,http://www.crunchbase.com/company/bojam,http://www.bojam.com,,,None,games_video,5.0,...,[],$0,[],[],"[{'description': 'Palo Alto Office', 'address1...","[{'embed_code': '<object id=""otv_o_557762"" hei...",[],[],[],NaN


In [11]:
data.columns

Index(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url',
       'blog_url', 'blog_feed_url', 'twitter_username', 'category_code',
       'number_of_employees', 'founded_year', 'founded_month', 'founded_day',
       'deadpooled_year', 'deadpooled_month', 'deadpooled_day',
       'deadpooled_url', 'tag_list', 'email_address', 'phone_number',
       'description', 'created_at', 'updated_at', 'overview', 'image',
       'products', 'relationships', 'competitions', 'providerships',
       'total_money_raised', 'funding_rounds', 'investments', 'offices',
       'video_embeds', 'screenshots', 'external_links', 'partners', 'ipo'],
      dtype='object')

In [12]:
def dropcolumn(df,column):
    df.drop(f"{column}", axis = 1, inplace = True)
    
    return df

In [13]:
columns=['permalink', 'crunchbase_url', 'homepage_url',
       'blog_url', 'blog_feed_url', 'twitter_username',
        'founded_year', 'founded_month', 'founded_day',
       'deadpooled_year', 'deadpooled_month', 'deadpooled_day',
       'deadpooled_url', 'tag_list', 'email_address', 'phone_number',
       'created_at', 'updated_at', 'image',
        'relationships', 'providerships', 'funding_rounds',
       'video_embeds', 'screenshots', 'external_links', 'partners', 'ipo','competitions','overview','_id','products']
for i in columns:
    dropcolumn(data,i)

In [33]:
data.sample(2)

,name,category_code,number_of_employees,description,total_money_raised,offices
751,Playlogic,games_video,NaN,GAME PUBLISHER,$19.3M,"[{'description': '', 'address1': 'Playlogic In..."
436,LorenzGames,games_video,3.0,FLASH ONLINE GAMES,NaN,[]


In [ ]:
data['description'].value_counts()

In [18]:
data['description'] = data['description'].str.upper()

In [21]:
data['description'] = data['description'].str.extract('(.*GAM.*)')

In [64]:
data.sample(10)

,name,category_code,number_of_employees,description,total_money_raised,offices
136,WonderHill,games_video,NaN,CASUAL GAMING,$7M,NaN
156,Bigpoint,games_video,500.0,DEVELOPER AND PROVIDER OF ONLINE GAMES,€420M,"{'description': 'Bigpoint Berlin', 'address1':..."
15,Zynga,games_video,115.0,NETWORK OF GAMING APPLICATIONS,$860M,"{'description': '', 'address1': '', 'address2'..."
36,Moblyng,games_video,NaN,HTML5 SOCIAL GAMES,$19.1M,"{'description': '', 'address1': '901 Marshall ..."
141,Playdom,games_video,NaN,GAMING NETWORK,$76M,"{'description': 'Palo Alto - Headquarters', 'a..."
97,Gamervision,games_video,NaN,GAME TRADING SITE,$10M,"{'description': '', 'address1': '4030 Skyron D..."
156,Bigpoint,games_video,500.0,DEVELOPER AND PROVIDER OF ONLINE GAMES,€420M,"{'description': 'Bigpoint Headquarters', 'addr..."
56,UltiZen,games_video,NaN,ART AND GAME DEVELOPMENT,$15.5M,"{'description': 'HQ', 'address1': '879 Zhong J..."
106,BASH Gaming,games_video,25.0,SOCIAL MOBILE CASINO GAMES,$1M,"{'description': 'India Office', 'address1': ''..."
15,Zynga,games_video,115.0,NETWORK OF GAMING APPLICATIONS,$860M,"{'description': '', 'address1': '', 'address2'..."


In [26]:
 data=data.dropna(subset=['description'])

In [30]:
data['total_money_raised'] = data['total_money_raised'].str.extract('(.*M|B)')

C:\Users\mataf\AppData\Local\Temp\ipykernel_19960\3121672454.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['total_money_raised'] = data['total_money_raised'].str.extract('(.*M|B)')


In [29]:
check=data['total_money_raised'].value_counts()
print(check.to_string())

$0        84
$3M        4
$1M        3
$95.1M     2
$1.5M      2
$6M        2
$15.5M     2
$1.1M      2
$7M        2
$7.5M      2
$19M       2
$59.5M     1
$80M       1
$25k       1
€850k      1
$550k      1
$5.8M      1
$15k       1
$36M       1
¥464M      1
$19.3M     1
$2M        1
$140k      1
$136M      1
$7.66M     1
$44.8M     1
€420M      1
$11.4M     1
$3.86M     1
$76M       1
$40M       1
$6.5M      1
€3M        1
$40.8M     1
$250k      1
€150k      1
$8.25M     1
€750k      1
€1.5M      1
$315k      1
€4M        1
$10M       1
$600k      1
$7.26M     1
$37.7M     1
$1.9M      1
$27M       1
$17.1M     1
$17.5M     1
$860M      1
$23.3M     1
$84M       1
$225k      1
$586M      1
$56.5M     1
$46.3M     1
$3.97M     1
$16.6M     1
$10.5M     1
$21M       1
$5.92M     1
$18M       1
$19.1M     1
$12M       1
$8M        1
$4.5M      1
$48.7M     1
$200k      1
$15M       1
$10.7M     1
$82.8M     1
$5.55M     1
$76.6M     1
$125M      1
$4M        1
€2.75M     1
$5.23M     1

In [56]:
 data=data.dropna(subset=['total_money_raised'])

In [ ]:
data.explode('offices')

In [37]:
data.reset_index(drop=True, inplace=True)

In [66]:
df_values=data['offices'].apply(pd.Series)

In [74]:
dropcolumn(df_values,'description')

,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,0
0,585 Broadway,Suite A,94063,Redwood City,CA,USA,37.484619,-122.206893,NaN
1,60 Broadway,,94111,San Francisco,CA,USA,37.787092,-122.399972,NaN
1,150 West Park Loop NW,,35806,Huntsville,AL,USA,NaN,NaN,NaN
2,2 Shaw Alley,4th Floor,94105,San Francisco,CA,USA,37.786942,-122.401245,NaN
3,,,02111,Boston,MA,USA,42.350274,-71.058768,NaN
...,...,...,...,...,...,...,...,...,...
157,632 W. 28th St.,8th Floor,10001,New York,NY,USA,40.752380,-74.005568,NaN
157,25 Bazel Street,PO Box 2645,49125,Petach-Tikva,None,ISR,NaN,NaN,NaN
157,685 Market Street,Suite 560,94105,San Francisco,CA,USA,37.787646,-122.402759,NaN
163,2401 4th Ave,Suite 810,98121,Seattle,WA,USA,47.616169,-122.345190,NaN


In [75]:
Test=pd.concat([data, df_values], axis=1)

In [79]:
Test.columns

Index([               'name',       'category_code', 'number_of_employees',
               'description',  'total_money_raised',             'offices',
                  'address1',            'address2',            'zip_code',
                      'city',          'state_code',        'country_code',
                  'latitude',           'longitude',                     0],
      dtype='object')

In [84]:
columns=['state_code','offices',0,'address2','zip_code','address1']
for i in columns:
    dropcolumn(Test,i)

In [90]:
Test = Test.iloc[:, :-1]

In [101]:
Test.sample(10)

,name,category_code,number_of_employees,description,total_money_raised,city,country_code,latitude,longitude
123,SCVNGR,games_video,60.0,"PART GAME, PART GAME PLATFORM",$40.8M,Boston,USA,42.340653,-71.068003
157,Exent,games_video,100.0,GAMES ON DEMAND LEADER,$3M,New York,USA,40.752380,-74.005568
33,Turbine,games_video,200.0,CREATES AND OPERATES ONLINE GAMES,$76.6M,Redwood City,USA,37.477652,-122.223384
5,OMGPOP,games_video,50.0,FREE ONLINE MULTIPLAYER GAME,$16.6M,New York,USA,40.723384,-74.001704
8,Rupture,games_video,25.0,GAMING SOCIAL NETWORK,$3M,San Francisco,USA,37.783898,-122.395234
16,GameDuell,games_video,200.0,CASUAL ONLINE GAMES,$17.5M,Berlin,DEU,52.513183,13.394149
32,Kabam,games_video,650.0,LEADER IN GAMING,$125M,San Francisco,USA,37.394456,-122.078531
3,MocoSpace,games_video,25.0,LARGEST MOBILE GAMES COMMUNITY IN N AMER,$10.5M,Boston,USA,42.350274,-71.058768
124,Owlient,games_video,40.0,BROWSER GAMES & COMMUNITY SERVICES,€3M,Paris,FRA,48.830809,2.303220
6,FlowPlay,games_video,30.0,VIRTUAL WORLD TECHNOLOGY AND GAMES,$3.97M,Seattle,USA,47.601532,-122.335943


In [ ]:
def dropna(df,subset):
    Test=Test.dropna(subset=[f'column'])
    

In [94]:
Test=Test.dropna(subset=['latitude'])

In [95]:
Test=Test.dropna(subset=['longitude'])

In [99]:
Test=Test.dropna(subset=['number_of_employees'])

In [102]:
check=Test['city'].value_counts()
print(check.to_string())

San Francisco    11
New York          4
Redwood City      2
Boston            2
Seattle           2
Madrid            1
Paris             1
Prague 8          1
Bangalore         1
Fremont           1
El Segundo        1
Westwood          1
Dundee            1
Boulder           1
London            1
Montreal          1
Beverly Hills     1
Los Angeles       1
Berlin            1
Hamburg           1


In [ ]:
#col=col.dropna(subset=['latitude'])

In [ ]:
#columna=Test['offices'].apply(lambda x:x['latitude'])

In [ ]:
#y=json.loads('Offices.json')